In [38]:
# Import the data processing libraries (note: you may have to install this!)
import pandas as pd
import altair as alt

alt.data_transformers.enable('default', max_rows=None) #disable max rows allowed for altair so we can work with the full dataset

# GET THE DATA

# Option 1: Local path (for cloned repos)
#data = pd.read_csv("../Data/ECommerceData.csv", encoding="ISO-8859-1") 

# Option 2: Online GitHub-hosted path (no cloning of repo required)
data = pd.read_csv(
    "https://raw.githubusercontent.com/lbeeler1/altair-sales-insights-dashboard/main/Data/ECommerceData.csv",
    encoding='ISO-8859-1' #needed due to special characters preventing parsing via utf-8
)

# CLEAN THE DATA IN PREP FOR VISUALIZATION

#check for missing values
data.isnull().sum()

missing_percent = (data['Description'].isnull().mean()) * 100
print(f"Percentage of missing descriptions: {missing_percent:.2f}%")

# Drop rows without product description since they make up only 0.27% of the dataset
data = data.dropna(subset=['Description'])

# Optionally drop rows with missing CustomerID if needed (not needed for this project which doesnt consider customers)
# data = data.dropna(subset=['CustomerID'])

# Remove returns or errors
data = data[(data['Quantity'] > 0) & (data['UnitPrice'] > 0)]

# Create a TotalSales column
data['TotalSales'] = (data['Quantity'] * data['UnitPrice']).round(2)

# Make sure InvoiceDate is datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Extract just the date part
data['Date'] = data['InvoiceDate'].dt.floor('D')

# change all instances of EIRE (Irish word for Ireland) to Ireland 
data['Country'] = data['Country'].replace('EIRE', 'Ireland')

# View cleaned data
data.head()

Percentage of missing descriptions: 0.27%


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSales,Date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01


In [40]:
# --------- Top Products Chart (Independent) ---------
top_products = (
    data.groupby('Description')['TotalSales']
    .sum()
    .reset_index()
    .sort_values('TotalSales', ascending=False)
    .head(10)
)

products_chart = alt.Chart(top_products).mark_bar().encode(
    x=alt.X('TotalSales:Q', title='Total Sales'),
    y=alt.Y('Description:N', sort='-x', title='Product'),
    tooltip=['Description', 'TotalSales']
).properties(
    width=1000,
    height=300,
    title='Top 10 Selling Products'
)

# --------- Filter to Top 6 Countries Excluding UK ---------
sales_by_country = (
    data.groupby('Country')['TotalSales']
    .sum()
    .reset_index()
    .sort_values('TotalSales', ascending=False)
)

top_countries = sales_by_country[sales_by_country['Country'] != 'United Kingdom'].head(6)['Country'].tolist()
filtered_data = data[data['Country'].isin(top_countries)]

# --------- Group Data by Date and Country ---------
sales_over_time = filtered_data.groupby(['Date', 'Country'])['TotalSales'].sum().reset_index()

# --------- Selection for Country Filtering ---------
country_selection = alt.selection_multi(fields=['Country'], bind='legend')

# --------- Bar Chart: Total Sales by Country ---------
bar = alt.Chart(
    filtered_data.groupby('Country')['TotalSales'].sum().reset_index()
).mark_bar().encode(
    y=alt.Y('Country:N', sort='-x', title='Country'),
    x=alt.X('TotalSales:Q', title='Total Sales'),
    color=alt.condition(country_selection, alt.Color('Country:N'), alt.value('lightgray')),
    tooltip=['Country', 'TotalSales']
).add_selection(
    country_selection
).properties(
    width=300,
    height=500,
    title='Total Sales by Country'
)

# --------- Scatter + Line Chart: Total Sales Over Time ---------
brush = alt.selection_interval(bind='scales', encodings=['x'])

line_chart = alt.Chart(sales_over_time).transform_filter(
    country_selection
).transform_aggregate(
    total_sales='sum(TotalSales)',
    groupby=['Date']
).mark_line(point=True).encode(
    x=alt.X('Date:T', title='Invoice Date'),
    y=alt.Y('total_sales:Q', title='Total Sales', scale=alt.Scale(zero=False)),
    tooltip=['Date:T', 'total_sales:Q']
).add_selection(
    brush
).properties(
    width=700,
    height=500,
    title='Total Sales Over Time (Filtered by Country)'
)

# --------- Combine into Dashboard ---------
dashboard = alt.vconcat(
    products_chart,
    alt.hconcat(bar, line_chart).resolve_scale(color='independent')
)

dashboard


/Users/Loseph/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/Loseph/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/Loseph/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.VConcatChart(...)